<div style="text-align: center;">
    <h1>TC's Capstone Project for Applied Data Science</h1>
    <h2>This notebook is being used for my Coursera's Applied Data Science Capstone Project</h2>
    <span><img src="https://upload.wikimedia.org/wikipedia/en/a/aa/Bart_Simpson_200px.png" alt="Bart Simpsons" /></span>
</div>

In [1]:
# 0. Installing Beautiful Soup 4 for scraping html/xml doc
!pip install beautifulsoup4
!pip install lxml
!easy_install -m  lxml


     |████████████████████████████████| 122kB 6.1MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 21.2MB/s eta 0:00:01     |██████████▌                     | 1.8MB 21.2MB/s eta 0:00:01
Searching for lxml
Best match: lxml 4.5.1

Using /home/jupyterlab/conda/envs/python/lib/python3.6/site-packages
Because this distribution was installed --multi-version, before you can
import modules from this package in an application, you will need to
'import pkg_resources' and then use a 'require()' call similar to one of
these examples, in order to select the desired version:

    pkg_resources.require("lxml")  # latest installed version
    pkg_resources.require("lxml==4.5.1")  # this exact version
    pkg_resources.require("lxml>=4.5.1")  # this version or higher

Processing dependencies for lxml
Finished processing dependencies for lxml


In [11]:
# Install python geocoder
!pip install geocoder

     |████████████████████████████████| 102kB 14.7MB/s ta 0:00:01
     |████████████████████████████████| 92kB 6.0MB/s eta 0:00:011
     |████████████████████████████████| 829kB 26.9MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [12]:
# 1. Import Library
import numpy as np 
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import lxml
import lxml.etree

# import geocoder
import geocoder 

### Remarks:
1. Beautiful Soup library is used to parse the HTML page into BeautifulSoup Object object for result manipulation
2. Default parser has been used in this case

In [3]:
# 2. Get the page on wiki
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(url)
soup = BeautifulSoup(res.content)
raw_tables_contents = soup.table.find_all("tr")

In [4]:
# 3. transforming data from tables_contens into df
# 3.1 Columns
columns_label = raw_tables_contents[0].get_text().replace(" ", "")
columns_label_list = list(filter(None, columns_label.split('\n')))
postal_codes_df = pd.DataFrame(columns=columns_label_list)

In [5]:
# 3.2 Actual data
# 3.2.1 Transform raw data and append into the df
raw_data = raw_tables_contents[1:]
for idx, row in enumerate(raw_data):
   data_row = list(filter(None, row.get_text().split('\n')))
   postal_codes_df.loc[idx] = data_row

In [6]:
# 3.2.2 Cleaning the df
# Remove row with Borough == 'Not assigned'
postal_codes_df = postal_codes_df[postal_codes_df.Borough != 'Not assigned']

### Remarks:
1. In the data table, if Neightborhood is 'Not assigned', Borough must be 'Not assigned' as well, so no extra efforts are spent to clean such data. <br/>
2. Neighborhoods with same PostalCode are combined into one row, delimited by ',' by default, so no extra efforts are spent to clean such data.

In [7]:
postal_codes_df[postal_codes_df.Neighborhood == 'Not assigned'].shape

(0, 3)

In [8]:
postal_codes_df.reset_index(drop=True, inplace=True)
postal_codes_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
postal_codes_df.shape

(103, 3)

In [16]:
# Get the csv for Latitude and Longitude
!wget -O Geospatial_data.csv http://cocl.us/Geospatial_data

--2020-06-16 20:35:34--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 158.85.108.86, 169.48.113.194, 158.85.108.83
Connecting to cocl.us (cocl.us)|158.85.108.86|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-06-16 20:35:34--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|158.85.108.86|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-06-16 20:35:35--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.29.197
Connecting to ibm.box.com (ibm.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-06-16 20:35:35--  https://ibm.box.com/public

In [31]:
geo_partial_data = pd.read_csv("Geospatial_data.csv", delimiter=",", names=['PostalCode', 'Latitude', 'Longitude'])
geo_partial_data = geo_partial_data.iloc[1:].set_index('PostalCode')
geo_partial_data.head()

,Latitude,Longitude
PostalCode,,
M1B,43.8066863,-79.1943534
M1C,43.7845351,-79.1604971
M1E,43.7635726,-79.1887115
M1G,43.7709921,-79.2169174
M1H,43.773136,-79.2394761


In [26]:
postal_codes_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [41]:
# Merge geo_partial_data with postal_codes_df
pc_geo_df = postal_codes_df.join(geo_partial_data, how='inner', on='PostalCode')
pc_geo_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.4647633
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623015,-79.3894938


In [47]:
pc_geo_df.loc[pc_geo_df['PostalCode']=='M9V']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.7394164,-79.5884369
